# Experiment 035D: Bonded State Decomposition

**AKIRA Project - Oscar Goldman - Shogu Research Group @ Datamutant.ai**

---

## Goal

Test whether complex ACTION DISCRIMINATIONS decompose into simpler AQ.

From `ACTION_QUANTA.md`:

```
AQ (pattern) -> enables DISCRIMINATION -> enables ACTION

Single AQ: Simple pattern -> Simple discrimination -> Simple action
Bonded state: Multiple AQ -> Complex discrimination -> Complex action
```

From `RADAR_ARRAY.md`:

```
BONDED STATE: "Anti-ship missile inbound"
  = AQ1 (CLOSING RAPIDLY) + AQ2 (CLOSE!) + AQ3 (SEA-SKIMMING) + ...
  -> Enables ACTION: "ENGAGE IMMEDIATELY"
```

---

## Hypothesis

If AQ theory is correct:
1. Complex action discriminations should show activation patterns containing their component AQ
2. Adding more AQ components should produce predictably combined patterns
3. The bonded state activation should be related to (but not simply sum of) component activations

---

## Key Insight: AQ are about ACTION, not MEANING

We test prompts that require the model to discriminate between ACTION alternatives:
- FLEE vs STAY
- URGENT vs NOT URGENT  
- ENGAGE vs WAIT
- ACT NOW vs DELAY

NOT prompts about understanding meaning (that's not what AQ do).

---

## 1. Setup

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install transformers torch numpy scikit-learn matplotlib seaborn scipy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
import warnings
from scipy import stats
from itertools import combinations

warnings.filterwarnings('ignore')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

In [ ]:
@dataclass
class ExperimentConfig:
    """Configuration for bonded state decomposition experiment."""
    model_name: str = "gpt2-medium"
    layers_to_probe: List[int] = field(default_factory=list)
    random_seed: int = 42
    
    def __post_init__(self) -> None:
        """Initialize layer indices based on model."""
        if not self.layers_to_probe:
            if "gpt2-medium" in self.model_name.lower():
                # GPT-2 Medium has 24 layers
                self.layers_to_probe = [0, 4, 8, 12, 16, 20, 23]
            elif "gpt2" in self.model_name.lower():
                self.layers_to_probe = [0, 3, 6, 9, 11]
            else:
                self.layers_to_probe = [0, 4, 8, 12, 16, 20, 23]
        
        np.random.seed(self.random_seed)
        torch.manual_seed(self.random_seed)


config = ExperimentConfig()
print(f"Model: {config.model_name}")
print(f"Layers to probe: {config.layers_to_probe}")

## 3. Model Loading

In [ ]:
print(f"Loading {config.model_name}...")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    output_hidden_states=True
)
model = model.to(DEVICE)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M parameters")
print(f"Number of layers: {model.config.n_layer}")

## 4. Action Discrimination Prompts

### Design Principle

Each prompt requires the model to discriminate between ACTION alternatives.
We structure prompts hierarchically:

- **Level 1**: Single AQ (one action discrimination)
- **Level 2**: Two-bond (two discriminations combined)
- **Level 3**: Three-bond (three discriminations)
- **Level 4**: Multi-bond (complex action requiring many AQ)

### AQ Components We Test

| AQ Component | Discrimination | Action Enabled |
|--------------|----------------|----------------|
| THREAT | Dangerous vs Safe | Respond vs Ignore |
| URGENCY | Immediate vs Later | Act now vs Wait |
| DIRECTION | Left/Right/Toward/Away | Move appropriately |
| PROXIMITY | Close vs Far | Prioritize response |
| AGENCY | Self vs Other | Who should act |
| CERTAINTY | Sure vs Unsure | Commit vs Investigate |

In [ ]:
# =============================================================================
# LEVEL 1: SINGLE AQ - One action discrimination
# =============================================================================

SINGLE_AQ_PROMPTS = {
    # THREAT discrimination (DANGEROUS vs SAFE)
    "threat": [
        "A snake is in front of you. You should",
        "There's a fire in the building. You should",
        "A car is speeding toward you. You should",
        "You hear a gunshot nearby. You should",
        "The bridge is collapsing. You should",
        "A wild bear appears. You should",
        "The alarm is ringing. You should",
        "Smoke is filling the room. You should",
    ],
    
    # URGENCY discrimination (NOW vs LATER)
    "urgency": [
        "The deadline is in 5 minutes. You should",
        "The last train leaves now. You should",
        "The patient is coding. You should",
        "The bomb timer shows 10 seconds. You should",
        "The auction ends in 3 seconds. You should",
        "The plane is boarding final call. You should",
        "The ice is cracking beneath you. You should",
        "The opportunity expires today. You should",
    ],
    
    # PROXIMITY discrimination (CLOSE vs FAR)
    "proximity": [
        "The exit is right next to you. You should",
        "Help is just around the corner. You should",
        "The destination is 5 steps away. You should",
        "Safety is within arm's reach. You should",
        "The goal is inches from you. You should",
        "The door is right there. You should",
        "The finish line is just ahead. You should",
        "The shelter is nearby. You should",
    ],
    
    # DIRECTION discrimination (LEFT/RIGHT/TOWARD/AWAY)
    "direction": [
        "The safe path is to your left. You should",
        "The exit is behind you. You should",
        "The danger is to your right. You should",
        "The noise came from above. You should",
        "The rescue team is approaching from the north. You should",
        "The threat is moving away from you. You should",
        "The opportunity is straight ahead. You should",
        "The obstacle is blocking your path forward. You should",
    ],
}

print(f"Single AQ categories: {list(SINGLE_AQ_PROMPTS.keys())}")
print(f"Prompts per category: {len(SINGLE_AQ_PROMPTS['threat'])}")

In [ ]:
# =============================================================================
# LEVEL 2: TWO-BOND - Two action discriminations combined
# =============================================================================

TWO_BOND_PROMPTS = {
    # THREAT + URGENCY
    "threat_urgency": [
        "A dangerous fire is spreading rapidly toward you. You should",
        "A venomous snake is about to strike. You should",
        "The building will collapse in seconds. You should",
        "A speeding car is seconds away from hitting you. You should",
        "The bomb will explode immediately. You should",
        "The flood water is rising fast around you. You should",
        "The predator is charging at you now. You should",
        "The gas leak will ignite any moment. You should",
    ],
    
    # THREAT + DIRECTION
    "threat_direction": [
        "A dangerous dog is blocking the path to your left. You should",
        "Fire is spreading from the right side. You should",
        "The attacker is approaching from behind. You should",
        "Falling debris is coming from above. You should",
        "The hazard is directly in front of you. You should",
        "Toxic gas is leaking from below. You should",
        "The threat is moving toward you from the north. You should",
        "Danger is closing in from all sides except the exit behind you. You should",
    ],
    
    # URGENCY + PROXIMITY
    "urgency_proximity": [
        "The closing door is right next to you and shutting now. You should",
        "The last lifeboat is departing and it's right beside you. You should",
        "The safety button is within reach and must be pressed immediately. You should",
        "The escape hatch is at your feet and closing fast. You should",
        "The final opportunity is in your hands and expires now. You should",
        "The rescue helicopter is hovering above you and leaving soon. You should",
        "The cure is on the table beside you and the patient is dying. You should",
        "The parachute release is at your fingertips and you're falling. You should",
    ],
    
    # DIRECTION + PROXIMITY
    "direction_proximity": [
        "The exit is just two steps to your left. You should",
        "Safety is immediately behind you, very close. You should",
        "The goal is directly ahead, only inches away. You should",
        "Help is right above you, within reach. You should",
        "The solution is to your right, arm's length away. You should",
        "The path forward is clear and the destination is near. You should",
        "The shelter is behind that nearby door. You should",
        "The weapon is on the floor to your immediate left. You should",
    ],
}

print(f"Two-bond categories: {list(TWO_BOND_PROMPTS.keys())}")
print(f"Prompts per category: {len(TWO_BOND_PROMPTS['threat_urgency'])}")

In [ ]:
# =============================================================================
# LEVEL 3: THREE-BOND - Three action discriminations combined
# =============================================================================

THREE_BOND_PROMPTS = {
    # THREAT + URGENCY + DIRECTION
    "threat_urgency_direction": [
        "A fire is rapidly spreading from the left and you must move right immediately. You should",
        "An attacker with a knife is rushing at you from behind, act now. You should",
        "The ceiling is collapsing fast directly above you, move away instantly. You should",
        "A car is about to hit you from the right, jump left now. You should",
        "Flood waters are rising rapidly from below, climb up immediately. You should",
        "A dangerous gas cloud is approaching fast from the north, flee south now. You should",
        "The explosion will happen in seconds to your left, run right immediately. You should",
        "A predator is charging from ahead, escape behind you this instant. You should",
    ],
    
    # THREAT + URGENCY + PROXIMITY
    "threat_urgency_proximity": [
        "A bomb right next to you will explode in 3 seconds. You should",
        "The venomous spider on your arm is about to bite. You should",
        "The fire beside you is spreading and will engulf you instantly. You should",
        "The collapsing beam directly above you is falling now. You should",
        "The knife at your throat is about to cut. You should",
        "The cracking ice beneath your feet will break any moment. You should",
        "The electrical wire touching your hand will short circuit now. You should",
        "The gas filling the small room you're in will ignite immediately. You should",
    ],
    
    # THREAT + DIRECTION + PROXIMITY
    "threat_direction_proximity": [
        "A snake is coiled to your immediate left, very close. You should",
        "Fire blocks the path directly ahead, but the exit is close behind you. You should",
        "A guard with a gun is standing right next to the door on your right. You should",
        "The poison gas is seeping in from directly below where you stand. You should",
        "Debris is falling from above, right where you're standing. You should",
        "The attacker is positioned just to your left, blocking that path. You should",
        "The hazard is directly in your path, but safety is close to the right. You should",
        "The threat is behind you at close range, the escape is ahead. You should",
    ],
    
    # URGENCY + DIRECTION + PROXIMITY  
    "urgency_direction_proximity": [
        "The last train leaving immediately is on the platform to your left, right there. You should",
        "The closing window of escape is directly ahead, just steps away, shutting now. You should",
        "The opportunity expires in seconds and it's right behind you, turn around. You should",
        "The rescue line is being pulled up now, it's directly above within reach. You should",
        "The door is closing fast, it's to your right, you can touch it. You should",
        "The auction ends now, the bid button is right in front of you. You should",
        "The helicopter is leaving immediately, the ladder is just above your head. You should",
        "The deadline is now, the submit button is right there on your screen. You should",
    ],
}

print(f"Three-bond categories: {list(THREE_BOND_PROMPTS.keys())}")
print(f"Prompts per category: {len(THREE_BOND_PROMPTS['threat_urgency_direction'])}")

In [ ]:
# =============================================================================
# LEVEL 4: FOUR-BOND - All four discriminations combined (full bonded state)
# =============================================================================

FOUR_BOND_PROMPTS = {
    # THREAT + URGENCY + DIRECTION + PROXIMITY (Complete bonded state)
    "full_bonded": [
        "A fire is rapidly engulfing the room from the left, the only exit is to your right just 3 feet away, move now or die. You should",
        "An armed attacker is charging at you from behind, 5 feet away, closing fast, the door ahead is within reach. You should",
        "The ceiling is collapsing immediately above you, safety is just one step to your left, you have a second. You should",
        "A speeding car is about to hit you from the right, 2 seconds away, the sidewalk to your left is one jump away. You should",
        "Flood water is rising rapidly around you, the ladder above is within arm's reach, climb now or drown. You should",
        "Toxic gas is pouring in fast from below, the escape hatch above is right there, open it immediately or suffocate. You should",
        "The bomb beside you will detonate in 5 seconds, the blast shield is to your right, two steps away, get behind it now. You should",
        "A predator is lunging at you from the front, the tree behind you is close enough to climb, turn and climb instantly or be caught. You should",
        "The sniper is targeting you from ahead, cover is to your left just feet away, dive there now or be shot. You should",
        "Lava is flowing toward you fast from behind, the safe rock ahead is one jump away, leap now or burn. You should",
    ]
}

print(f"Four-bond (full) prompts: {len(FOUR_BOND_PROMPTS['full_bonded'])}")

In [ ]:
# =============================================================================
# CONTROL: Non-action prompts (should NOT show strong AQ patterns)
# =============================================================================

CONTROL_PROMPTS = [
    "The sky is blue today. It is",
    "Mathematics involves numbers and equations. It is",
    "The capital of France is Paris. It is",
    "Water is composed of hydrogen and oxygen. It is",
    "Books contain written words. They are",
    "The sun rises in the east. It is",
    "Trees have leaves and branches. They are",
    "Music consists of organized sounds. It is",
]

print(f"Control prompts: {len(CONTROL_PROMPTS)}")

## 5. Activation Capture

In [ ]:
def get_activation(prompt: str, model: nn.Module, tokenizer: AutoTokenizer, 
                   layers: List[int]) -> Dict[int, np.ndarray]:
    """Get last token activation at specified layers.
    
    Args:
        prompt: Input text
        model: The model
        tokenizer: The tokenizer
        layers: List of layer indices
        
    Returns:
        Dict mapping layer index to activation vector
    """
    assert prompt is not None, "Prompt required"
    assert len(prompt) > 0, "Prompt cannot be empty"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    activations = {}
    for layer_idx in layers:
        # Get last token activation
        h = outputs.hidden_states[layer_idx][0, -1, :].cpu().numpy()
        activations[layer_idx] = h
    
    return activations


def get_category_activations(prompts: List[str], model: nn.Module, 
                             tokenizer: AutoTokenizer, layers: List[int]) -> Dict[int, np.ndarray]:
    """Get averaged activation for a category of prompts.
    
    Args:
        prompts: List of prompts in category
        model: The model
        tokenizer: The tokenizer  
        layers: List of layer indices
        
    Returns:
        Dict mapping layer index to averaged activation vector
    """
    all_activations = {layer: [] for layer in layers}
    
    for prompt in prompts:
        acts = get_activation(prompt, model, tokenizer, layers)
        for layer in layers:
            all_activations[layer].append(acts[layer])
    
    # Average across prompts
    averaged = {}
    for layer in layers:
        averaged[layer] = np.mean(all_activations[layer], axis=0)
    
    return averaged


print("Activation functions ready")

## 6. Capture All Activations

In [ ]:
print("Capturing activations for all categories...")
print("=" * 60)

# Store all activations
single_aq_activations = {}
two_bond_activations = {}
three_bond_activations = {}
four_bond_activations = {}
control_activations = None

# Single AQ
print("\nLevel 1: Single AQ...")
for category, prompts in SINGLE_AQ_PROMPTS.items():
    single_aq_activations[category] = get_category_activations(
        prompts, model, tokenizer, config.layers_to_probe
    )
    print(f"  {category}: {len(prompts)} prompts")

# Two-bond
print("\nLevel 2: Two-bond...")
for category, prompts in TWO_BOND_PROMPTS.items():
    two_bond_activations[category] = get_category_activations(
        prompts, model, tokenizer, config.layers_to_probe
    )
    print(f"  {category}: {len(prompts)} prompts")

# Three-bond
print("\nLevel 3: Three-bond...")
for category, prompts in THREE_BOND_PROMPTS.items():
    three_bond_activations[category] = get_category_activations(
        prompts, model, tokenizer, config.layers_to_probe
    )
    print(f"  {category}: {len(prompts)} prompts")

# Four-bond
print("\nLevel 4: Four-bond (full bonded state)...")
for category, prompts in FOUR_BOND_PROMPTS.items():
    four_bond_activations[category] = get_category_activations(
        prompts, model, tokenizer, config.layers_to_probe
    )
    print(f"  {category}: {len(prompts)} prompts")

# Control
print("\nControl (non-action)...")
control_activations = get_category_activations(
    CONTROL_PROMPTS, model, tokenizer, config.layers_to_probe
)
print(f"  control: {len(CONTROL_PROMPTS)} prompts")

print("\n" + "=" * 60)
print("All activations captured.")

## 7. Analysis: Do Bonded States Contain Component AQ?

In [ ]:
def compute_similarity_to_components(bonded_activation: np.ndarray,
                                     component_activations: List[np.ndarray]) -> Dict[str, float]:
    """Compute how similar a bonded state is to its components.
    
    Args:
        bonded_activation: The activation of the bonded state
        component_activations: List of component AQ activations
        
    Returns:
        Dict with similarity metrics
    """
    # Individual similarities
    individual_sims = []
    for comp in component_activations:
        sim = cosine_similarity([bonded_activation], [comp])[0, 0]
        individual_sims.append(sim)
    
    # Average of components
    avg_components = np.mean(component_activations, axis=0)
    sim_to_avg = cosine_similarity([bonded_activation], [avg_components])[0, 0]
    
    # Sum of components (normalized)
    sum_components = np.sum(component_activations, axis=0)
    sum_components = sum_components / np.linalg.norm(sum_components)
    bonded_norm = bonded_activation / np.linalg.norm(bonded_activation)
    sim_to_sum = cosine_similarity([bonded_norm], [sum_components])[0, 0]
    
    return {
        "individual_sims": individual_sims,
        "mean_individual_sim": np.mean(individual_sims),
        "sim_to_avg_components": sim_to_avg,
        "sim_to_sum_components": sim_to_sum
    }


print("Analysis functions ready")

In [ ]:
# Analyze two-bond compositions
print("ANALYSIS: TWO-BOND DECOMPOSITION")
print("=" * 70)
print("\nQuestion: Does 'threat_urgency' contain signatures of 'threat' AND 'urgency'?\n")

two_bond_analysis = {}

# Define which single AQ compose each two-bond
two_bond_components = {
    "threat_urgency": ["threat", "urgency"],
    "threat_direction": ["threat", "direction"],
    "urgency_proximity": ["urgency", "proximity"],
    "direction_proximity": ["direction", "proximity"],
}

for layer in config.layers_to_probe:
    print(f"\nLayer {layer}:")
    print("-" * 50)
    
    for bonded_name, component_names in two_bond_components.items():
        # Get bonded activation
        bonded_act = two_bond_activations[bonded_name][layer]
        
        # Get component activations
        component_acts = [single_aq_activations[c][layer] for c in component_names]
        
        # Compute similarities
        result = compute_similarity_to_components(bonded_act, component_acts)
        
        # Also compute similarity to control
        control_sim = cosine_similarity([bonded_act], [control_activations[layer]])[0, 0]
        
        print(f"  {bonded_name}:")
        for i, comp_name in enumerate(component_names):
            print(f"    -> {comp_name}: {result['individual_sims'][i]:.3f}")
        print(f"    Mean component sim: {result['mean_individual_sim']:.3f}")
        print(f"    Control sim: {control_sim:.3f}")
        print(f"    Ratio (component/control): {result['mean_individual_sim']/control_sim:.2f}x")
        
        if layer not in two_bond_analysis:
            two_bond_analysis[layer] = {}
        two_bond_analysis[layer][bonded_name] = {
            "component_sims": result["individual_sims"],
            "mean_component_sim": result["mean_individual_sim"],
            "control_sim": control_sim,
            "ratio": result["mean_individual_sim"] / control_sim
        }

In [ ]:
# Analyze three-bond compositions
print("\nANALYSIS: THREE-BOND DECOMPOSITION")
print("=" * 70)
print("\nQuestion: Do three-bond states contain all three component AQ signatures?\n")

three_bond_analysis = {}

# Define components
three_bond_components = {
    "threat_urgency_direction": ["threat", "urgency", "direction"],
    "threat_urgency_proximity": ["threat", "urgency", "proximity"],
    "threat_direction_proximity": ["threat", "direction", "proximity"],
    "urgency_direction_proximity": ["urgency", "direction", "proximity"],
}

for layer in config.layers_to_probe:
    print(f"\nLayer {layer}:")
    print("-" * 50)
    
    for bonded_name, component_names in three_bond_components.items():
        bonded_act = three_bond_activations[bonded_name][layer]
        component_acts = [single_aq_activations[c][layer] for c in component_names]
        
        result = compute_similarity_to_components(bonded_act, component_acts)
        control_sim = cosine_similarity([bonded_act], [control_activations[layer]])[0, 0]
        
        print(f"  {bonded_name}:")
        for i, comp_name in enumerate(component_names):
            print(f"    -> {comp_name}: {result['individual_sims'][i]:.3f}")
        print(f"    Mean component sim: {result['mean_individual_sim']:.3f}")
        print(f"    Control sim: {control_sim:.3f}")
        print(f"    Ratio: {result['mean_individual_sim']/control_sim:.2f}x")
        
        if layer not in three_bond_analysis:
            three_bond_analysis[layer] = {}
        three_bond_analysis[layer][bonded_name] = {
            "component_sims": result["individual_sims"],
            "mean_component_sim": result["mean_individual_sim"],
            "control_sim": control_sim,
            "ratio": result["mean_individual_sim"] / control_sim
        }

In [ ]:
# Analyze four-bond (full bonded state)
print("\nANALYSIS: FOUR-BOND (FULL BONDED STATE) DECOMPOSITION")
print("=" * 70)
print("\nQuestion: Does the full bonded state contain ALL four component AQ?\n")

four_bond_analysis = {}

all_single_aq = ["threat", "urgency", "direction", "proximity"]

for layer in config.layers_to_probe:
    print(f"\nLayer {layer}:")
    print("-" * 50)
    
    bonded_act = four_bond_activations["full_bonded"][layer]
    component_acts = [single_aq_activations[c][layer] for c in all_single_aq]
    
    result = compute_similarity_to_components(bonded_act, component_acts)
    control_sim = cosine_similarity([bonded_act], [control_activations[layer]])[0, 0]
    
    print(f"  Full bonded state (THREAT + URGENCY + DIRECTION + PROXIMITY):")
    for i, comp_name in enumerate(all_single_aq):
        print(f"    -> {comp_name}: {result['individual_sims'][i]:.3f}")
    print(f"    Mean component sim: {result['mean_individual_sim']:.3f}")
    print(f"    Sim to sum of components: {result['sim_to_sum_components']:.3f}")
    print(f"    Control sim: {control_sim:.3f}")
    print(f"    Ratio (mean/control): {result['mean_individual_sim']/control_sim:.2f}x")
    
    four_bond_analysis[layer] = {
        "component_sims": result["individual_sims"],
        "mean_component_sim": result["mean_individual_sim"],
        "sim_to_sum": result["sim_to_sum_components"],
        "control_sim": control_sim,
        "ratio": result["mean_individual_sim"] / control_sim
    }

## 8. Visualization

In [ ]:
# Plot component similarities across layers
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Two-bond decomposition
ax = axes[0, 0]
for bonded_name in two_bond_components.keys():
    ratios = [two_bond_analysis[layer][bonded_name]["ratio"] for layer in config.layers_to_probe]
    ax.plot(config.layers_to_probe, ratios, 'o-', label=bonded_name, markersize=8)
ax.axhline(y=1.0, color='k', linestyle='--', alpha=0.5, label='Control baseline')
ax.set_xlabel("Layer")
ax.set_ylabel("Component/Control Similarity Ratio")
ax.set_title("Two-Bond: Component Similarity vs Control")
ax.legend(loc='best', fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Three-bond decomposition
ax = axes[0, 1]
for bonded_name in three_bond_components.keys():
    ratios = [three_bond_analysis[layer][bonded_name]["ratio"] for layer in config.layers_to_probe]
    ax.plot(config.layers_to_probe, ratios, 'o-', label=bonded_name.replace("_", "+"), markersize=8)
ax.axhline(y=1.0, color='k', linestyle='--', alpha=0.5, label='Control baseline')
ax.set_xlabel("Layer")
ax.set_ylabel("Component/Control Similarity Ratio")
ax.set_title("Three-Bond: Component Similarity vs Control")
ax.legend(loc='best', fontsize=7)
ax.grid(True, alpha=0.3)

# Plot 3: Four-bond (full) decomposition by component
ax = axes[1, 0]
for i, comp_name in enumerate(all_single_aq):
    sims = [four_bond_analysis[layer]["component_sims"][i] for layer in config.layers_to_probe]
    ax.plot(config.layers_to_probe, sims, 'o-', label=comp_name.upper(), markersize=8)
control_sims = [four_bond_analysis[layer]["control_sim"] for layer in config.layers_to_probe]
ax.plot(config.layers_to_probe, control_sims, 's--', label='Control', color='gray', markersize=6)
ax.set_xlabel("Layer")
ax.set_ylabel("Cosine Similarity")
ax.set_title("Full Bonded State: Similarity to Each Component AQ")
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Plot 4: Four-bond ratio across layers
ax = axes[1, 1]
ratios = [four_bond_analysis[layer]["ratio"] for layer in config.layers_to_probe]
ax.bar(config.layers_to_probe, ratios, color='steelblue', alpha=0.7)
ax.axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='Control baseline')
ax.set_xlabel("Layer")
ax.set_ylabel("Component/Control Ratio")
ax.set_title("Full Bonded State: Component Similarity Ratio by Layer")
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle("035D: Bonded State Decomposition Analysis", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("bonded_state_decomposition.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved: bonded_state_decomposition.png")

In [ ]:
# Heatmap: Similarity matrix between all categories at best layer
# Find best layer (highest mean ratio)
best_layer = max(config.layers_to_probe, 
                 key=lambda l: four_bond_analysis[l]["ratio"])
print(f"Best layer for decomposition: {best_layer}")

# Collect all activations at best layer
all_categories = {}
for cat in SINGLE_AQ_PROMPTS.keys():
    all_categories[f"1_{cat}"] = single_aq_activations[cat][best_layer]
for cat in TWO_BOND_PROMPTS.keys():
    all_categories[f"2_{cat}"] = two_bond_activations[cat][best_layer]
for cat in THREE_BOND_PROMPTS.keys():
    all_categories[f"3_{cat}"] = three_bond_activations[cat][best_layer]
all_categories["4_full_bonded"] = four_bond_activations["full_bonded"][best_layer]
all_categories["0_control"] = control_activations[best_layer]

# Compute similarity matrix
category_names = sorted(all_categories.keys())
n_cats = len(category_names)
sim_matrix = np.zeros((n_cats, n_cats))

for i, name1 in enumerate(category_names):
    for j, name2 in enumerate(category_names):
        sim = cosine_similarity([all_categories[name1]], [all_categories[name2]])[0, 0]
        sim_matrix[i, j] = sim

# Plot
plt.figure(figsize=(14, 12))
sns.heatmap(sim_matrix, xticklabels=category_names, yticklabels=category_names,
            annot=True, fmt=".2f", cmap="RdYlGn", center=0.5,
            annot_kws={"size": 7})
plt.title(f"Similarity Matrix: All Categories (Layer {best_layer})\nBonded states should be similar to their components", 
          fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()
plt.savefig("category_similarity_matrix.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved: category_similarity_matrix.png")

## 9. Statistical Analysis

In [ ]:
# Test: Are bonded states significantly more similar to components than to control?
print("STATISTICAL ANALYSIS: Component Similarity vs Control")
print("=" * 70)
print("\nH0: Bonded states are equally similar to components and control")
print("H1: Bonded states are MORE similar to their components than to control\n")

# Collect all component similarities and control similarities
all_component_sims = []
all_control_sims = []

# Two-bond
for layer in config.layers_to_probe:
    for bonded_name in two_bond_components.keys():
        all_component_sims.extend(two_bond_analysis[layer][bonded_name]["component_sims"])
        all_control_sims.append(two_bond_analysis[layer][bonded_name]["control_sim"])

# Three-bond
for layer in config.layers_to_probe:
    for bonded_name in three_bond_components.keys():
        all_component_sims.extend(three_bond_analysis[layer][bonded_name]["component_sims"])
        all_control_sims.append(three_bond_analysis[layer][bonded_name]["control_sim"])

# Four-bond
for layer in config.layers_to_probe:
    all_component_sims.extend(four_bond_analysis[layer]["component_sims"])
    all_control_sims.append(four_bond_analysis[layer]["control_sim"])

# T-test
t_stat, p_value = stats.ttest_ind(all_component_sims, all_control_sims)

print(f"Component similarities: mean={np.mean(all_component_sims):.4f}, std={np.std(all_component_sims):.4f}")
print(f"Control similarities:   mean={np.mean(all_control_sims):.4f}, std={np.std(all_control_sims):.4f}")
print(f"\nT-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.6f}")

if p_value < 0.001:
    print("\nResult: HIGHLY SIGNIFICANT (p < 0.001) ***")
    print("Bonded states ARE more similar to their component AQ than to control.")
elif p_value < 0.01:
    print("\nResult: VERY SIGNIFICANT (p < 0.01) **")
elif p_value < 0.05:
    print("\nResult: SIGNIFICANT (p < 0.05) *")
else:
    print("\nResult: NOT SIGNIFICANT (p >= 0.05)")

In [ ]:
# Visualize the distribution
fig, ax = plt.subplots(figsize=(10, 6))

# Histogram
ax.hist(all_component_sims, bins=20, alpha=0.7, label=f'Component (n={len(all_component_sims)})', color='green')
ax.hist(all_control_sims, bins=20, alpha=0.7, label=f'Control (n={len(all_control_sims)})', color='red')

# Add means
ax.axvline(np.mean(all_component_sims), color='darkgreen', linestyle='--', linewidth=2, label=f'Component mean: {np.mean(all_component_sims):.3f}')
ax.axvline(np.mean(all_control_sims), color='darkred', linestyle='--', linewidth=2, label=f'Control mean: {np.mean(all_control_sims):.3f}')

ax.set_xlabel("Cosine Similarity")
ax.set_ylabel("Count")
ax.set_title(f"Distribution of Similarities: Components vs Control\n(p={p_value:.2e})")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("similarity_distribution.png", dpi=150, bbox_inches='tight')
plt.show()
print("Saved: similarity_distribution.png")

## 10. Summary and Conclusions

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 035D: BONDED STATE DECOMPOSITION SUMMARY")
print("=" * 70)

print(f"\nModel: {config.model_name}")
print(f"Layers probed: {config.layers_to_probe}")

print(f"\nPrompt counts:")
print(f"  Single AQ (Level 1): {sum(len(p) for p in SINGLE_AQ_PROMPTS.values())} prompts across {len(SINGLE_AQ_PROMPTS)} categories")
print(f"  Two-bond (Level 2): {sum(len(p) for p in TWO_BOND_PROMPTS.values())} prompts across {len(TWO_BOND_PROMPTS)} categories")
print(f"  Three-bond (Level 3): {sum(len(p) for p in THREE_BOND_PROMPTS.values())} prompts across {len(THREE_BOND_PROMPTS)} categories")
print(f"  Four-bond (Level 4): {len(FOUR_BOND_PROMPTS['full_bonded'])} prompts")
print(f"  Control: {len(CONTROL_PROMPTS)} prompts")

print(f"\nKey Results:")
print(f"  Component vs Control similarity: p = {p_value:.2e}")
print(f"  Mean component similarity: {np.mean(all_component_sims):.4f}")
print(f"  Mean control similarity: {np.mean(all_control_sims):.4f}")
print(f"  Ratio: {np.mean(all_component_sims)/np.mean(all_control_sims):.2f}x")

print(f"\nBest layer for decomposition: {best_layer}")
print(f"  Four-bond ratio at best layer: {four_bond_analysis[best_layer]['ratio']:.2f}x")

# Evidence assessment
evidence_score = 0
evidence_criteria = []

if p_value < 0.05:
    evidence_score += 1
    evidence_criteria.append("Component similarity > Control (p<0.05)")

if np.mean(all_component_sims) / np.mean(all_control_sims) > 1.1:
    evidence_score += 1
    evidence_criteria.append("Component/Control ratio > 1.1")

if four_bond_analysis[best_layer]["ratio"] > 1.2:
    evidence_score += 1
    evidence_criteria.append("Full bonded state shows strong component signatures")

print(f"\nEvidence Score: {evidence_score}/3")
for crit in evidence_criteria:
    print(f"  [x] {crit}")

## 11. Interpretation in AQ Framework

In [ ]:
print("""
INTERPRETATION IN AQ FRAMEWORK
==============================

From ACTION_QUANTA.md:

  Single AQ: Simple pattern -> Simple discrimination -> Simple action
  Bonded state: Multiple AQ -> Complex discrimination -> Complex action

From RADAR_ARRAY.md:

  BONDED STATE: "Anti-ship missile inbound"
    = AQ1 (CLOSING RAPIDLY) + AQ2 (CLOSE!) + AQ3 (SEA-SKIMMING) + ...
    -> Enables ACTION: "ENGAGE IMMEDIATELY"

This experiment tested:
  - Do bonded states (complex action prompts) contain signatures of component AQ?
  - Is the relationship stronger than random (control)?

If component similarity > control similarity:
  -> Bonded states DO contain their component AQ
  -> Complex action discriminations ARE composed of simpler ones
  -> The AQ framework correctly describes how the model represents actions

If component similarity = control similarity:
  -> Bonded states don't preserve component structure
  -> Complex actions may be represented holistically, not compositionally
  -> The AQ bonding hypothesis would need revision
""")

if p_value < 0.05 and np.mean(all_component_sims) > np.mean(all_control_sims):
    print("CONCLUSION: Evidence SUPPORTS the AQ bonding hypothesis.")
    print("Complex action discriminations contain detectable signatures of their component AQ.")
else:
    print("CONCLUSION: Evidence does NOT support the AQ bonding hypothesis.")
    print("Complex action discriminations may not be compositional in the expected way.")

In [ ]:
# Save results
import json

results = {
    "config": {
        "model_name": config.model_name,
        "layers_probed": config.layers_to_probe
    },
    "statistics": {
        "t_statistic": float(t_stat),
        "p_value": float(p_value),
        "mean_component_similarity": float(np.mean(all_component_sims)),
        "mean_control_similarity": float(np.mean(all_control_sims)),
        "ratio": float(np.mean(all_component_sims) / np.mean(all_control_sims))
    },
    "best_layer": best_layer,
    "four_bond_analysis": {str(k): {kk: float(vv) if isinstance(vv, (int, float, np.floating)) else [float(x) for x in vv] 
                                     for kk, vv in v.items()} 
                           for k, v in four_bond_analysis.items()},
    "evidence_score": evidence_score,
    "evidence_criteria": evidence_criteria
}

with open("035D_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("Results saved to 035D_results.json")